#####Libraries
***

In [1]:
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 9.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
--2022-09-19 12:11:48--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-19 12:11:48--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownlo

#####Load Dataset
***

In [2]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
  #άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-03-01','2022-07-01')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-03-01','2022-07-01')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [3]:
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,101.758369,72.935371,123.264778,109.157562,120.793571,223.396744,157.307007,169.428802,174.066833,136.349167,...,82.900169,273.630005,232.818268,33.455544,118.117958,326.841492,302.489990,214.501099,165.157501,52.391731
2021-03-02,101.232719,72.688164,123.596596,110.222755,116.100159,220.188339,154.726501,170.577301,173.244690,136.879944,...,83.274345,269.190002,232.127426,32.895824,115.247543,326.831726,293.420013,213.649567,164.504974,52.085178
2021-03-03,100.162666,72.564560,124.962952,109.597328,114.064705,218.140457,150.250000,173.620926,174.624695,140.163101,...,83.601753,255.059998,232.424866,31.874582,108.791580,325.103027,294.929993,212.738602,162.952621,52.503204
2021-03-04,99.167709,73.448799,125.001991,107.887169,106.950462,211.850494,148.878494,170.414627,168.527237,140.251556,...,82.853378,239.070007,229.978073,31.324680,109.741837,326.021118,288.600006,209.421509,159.828232,54.537560
2021-03-05,100.153275,75.017609,128.749710,110.730919,112.098389,216.784943,150.022995,170.031754,173.499176,144.822433,...,84.808495,239.050003,236.905838,32.345921,110.829277,339.000977,300.959991,213.293106,166.225327,56.599789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,150.928238,92.870499,172.150925,99.170822,97.111656,248.137924,116.459999,193.575409,164.681396,145.605087,...,96.279518,77.680000,281.488770,35.017067,77.258453,494.089691,248.289993,205.146637,172.702087,86.063782
2022-06-27,151.720825,94.239746,172.300293,98.812370,97.361015,246.087448,113.220001,197.166199,166.274918,143.752029,...,95.824852,75.940002,281.329712,35.186615,76.072624,504.078339,251.990005,203.200089,173.810226,88.173286
2022-06-28,151.076859,94.338966,172.290344,97.975998,94.777618,242.633484,107.400002,195.470001,163.456360,141.659866,...,96.546379,71.820000,278.745331,34.418652,72.525093,506.849640,247.990005,197.769714,168.658813,90.619514


In [9]:
rd1= rd.pct_change()
rd1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-02,-0.005166,-0.003389,0.002692,0.009758,-0.038855,-0.014362,-0.016404,0.006779,-0.004723,0.003893,...,0.004514,-0.016226,-0.002967,-0.016730,-0.024301,-0.000030,-0.029984,-0.003970,-0.003951,-0.005851
2021-03-03,-0.010570,-0.001700,0.011055,-0.005674,-0.017532,-0.009301,-0.028932,0.017843,0.007966,0.023986,...,0.003932,-0.052491,0.001281,-0.031045,-0.056018,-0.005289,0.005146,-0.004264,-0.009437,0.008026
2021-03-04,-0.009933,0.012186,0.000312,-0.015604,-0.062370,-0.028834,-0.009128,-0.018467,-0.034918,0.000631,...,-0.008952,-0.062691,-0.010527,-0.017252,0.008735,0.002824,-0.021463,-0.015592,-0.019174,0.038747
2021-03-05,0.009938,0.021359,0.029981,0.026359,0.048134,0.023292,0.007687,-0.002247,0.029502,0.032591,...,0.023597,-0.000084,0.030124,0.032602,0.009909,0.039813,0.042827,0.018487,0.040025,0.037813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,0.019338,0.012877,0.020968,0.028819,0.043739,0.040616,0.035752,-0.006820,0.063412,0.031768,...,0.053195,0.052432,0.040260,0.028111,0.047986,-0.008343,0.052790,0.045108,0.011815,0.019833
2022-06-27,0.005251,0.014744,0.000868,-0.003614,0.002568,-0.008263,-0.027821,0.018550,0.009676,-0.012727,...,-0.004722,-0.022400,-0.000565,0.004842,-0.015349,0.020216,0.014902,-0.009489,0.006416,0.024511
2022-06-28,-0.004244,0.001053,-0.000058,-0.008464,-0.026534,-0.014036,-0.051404,-0.008603,-0.016951,-0.014554,...,0.007530,-0.054253,-0.009186,-0.021825,-0.046633,0.005498,-0.015874,-0.026724,-0.029638,0.027743


In [10]:
#prepare the ranks for next month (20 working days later)
rd1_next_month = rd1.shift(-22)
rd1_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,0.013391,-0.000354,-0.020316,0.002634,0.053960,-0.001503,0.012696,-0.015264,-0.008262,-0.011324,...,-0.010643,0.026634,-0.025483,0.014741,-0.003904,-0.003989,-0.011704,-0.000802,0.004206,-0.015170
2021-03-02,0.002772,0.009917,0.003738,0.016956,0.059281,0.012734,0.021628,0.012953,0.024068,0.022130,...,0.017673,0.019354,0.013317,0.011912,0.005004,-0.013438,0.007288,0.024229,0.017658,0.027942
2021-03-03,-0.021931,0.008768,0.003443,0.017847,0.010811,0.012191,0.020794,-0.004869,0.015313,0.010306,...,0.000431,0.014058,-0.000398,0.016078,0.020496,0.008472,-0.000543,0.010053,0.009677,-0.015508
2021-03-04,-0.007160,0.005563,0.004272,0.011305,-0.024537,-0.007764,-0.000902,0.002742,0.007122,0.000616,...,0.003127,0.008645,0.010239,-0.002826,0.004228,-0.015695,-0.018400,-0.001780,-0.003635,-0.002832
2021-03-05,-0.001613,-0.000922,0.015202,-0.001065,-0.002866,0.007021,0.017237,0.006863,-0.008892,0.006706,...,0.009781,0.009519,0.008597,-0.022953,-0.000729,0.003485,-0.010540,0.002836,-0.004976,0.004437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#first: ranks assigned in order they appear in the array
rd2=rd1_next_month.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,41,28,5,31,45,25,40,7,19,14,...,16,44,2,42,23,22,13,27,34,8
2021-03-02,13,21,15,29,45,24,36,25,41,37,...,32,33,26,23,16,3,17,42,31,43
2021-03-03,2,18,13,37,22,24,39,5,33,21,...,11,30,9,34,38,17,7,20,19,3
2021-03-04,14,36,31,41,1,12,22,28,38,26,...,29,39,40,19,30,6,5,21,16,18
2021-03-05,14,18,42,17,11,35,44,34,4,33,...,39,38,37,1,19,27,2,26,8,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,13,6,15,20,33,31,28,3,43,23,...,41,39,30,19,36,1,40,34,5,14
2022-05-24,30,39,20,17,23,10,1,41,36,6,...,14,2,19,27,5,43,40,8,32,45
2022-05-25,36,40,39,31,11,23,3,30,17,22,...,43,1,29,15,4,42,20,10,9,44


In [12]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,5,4,1,4,5,3,5,1,3,2,...,2,5,1,5,3,3,2,3,4,1
2021-03-02,2,3,2,4,5,3,4,3,5,5,...,4,4,3,3,2,1,2,5,4,5
2021-03-03,1,2,2,5,3,3,5,1,4,3,...,2,4,1,4,5,2,1,3,3,1
2021-03-04,2,4,4,5,1,2,3,4,5,3,...,4,5,5,3,4,1,1,3,2,2
2021-03-05,2,2,5,2,2,4,5,4,1,4,...,5,5,5,1,3,3,1,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,2,1,2,3,4,4,4,1,5,3,...,5,5,4,3,4,1,5,4,1,2
2022-05-24,4,5,3,2,3,2,1,5,4,1,...,2,1,3,3,1,5,5,1,4,5
2022-05-25,4,5,5,4,2,3,1,4,2,3,...,5,1,4,2,1,5,3,2,1,5


In [13]:
#for absolute rank feature to use 
rd2_feature=rd1.rank(axis=1,method='first')
rd2_feature=rd2_feature.dropna()
rd2_feature.astype(int)
jk_feature=rd2_feature.copy()
jk_feature.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-02,25,30,39,45,1,12,10,42,26,40,...,41,11,33,9,4,37,3,28,29,21
2021-03-03,16,28,36,24,11,18,6,39,33,43,...,31,2,30,4,1,25,32,27,17,34
2021-03-04,24,40,32,19,2,6,25,16,5,33,...,26,1,23,17,37,34,11,20,12,45
2021-03-05,8,16,28,23,43,19,6,1,27,32,...,20,3,29,33,7,37,41,13,39,34
2021-03-08,9,34,28,41,1,22,7,43,42,33,...,38,2,29,8,45,24,19,37,35,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,13,6,15,20,33,31,28,3,43,23,...,41,39,30,19,36,1,40,34,5,14
2022-06-27,30,39,20,17,23,10,1,41,36,6,...,14,2,19,27,5,43,40,8,32,45
2022-06-28,36,40,39,31,11,23,3,30,17,22,...,43,1,29,15,4,42,20,10,9,44


In [14]:
r5_next_month=rd2_feature.copy()
rd3_next_month=rank_data(r5_next_month)
rd3_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-02,3,4,5,5,1,2,2,5,3,5,...,5,2,4,1,1,5,1,4,4,3
2021-03-03,2,4,4,3,2,2,1,5,4,5,...,4,1,4,1,1,3,4,3,2,4
2021-03-04,3,5,4,3,1,1,3,2,1,4,...,3,1,3,2,5,4,2,3,2,5
2021-03-05,1,2,4,3,5,3,1,1,3,4,...,3,1,4,4,1,5,5,2,5,4
2021-03-08,1,4,4,5,1,3,1,5,5,4,...,5,1,4,1,5,3,3,5,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,2,1,2,3,4,4,4,1,5,3,...,5,5,4,3,4,1,5,4,1,2
2022-06-27,4,5,3,2,3,2,1,5,4,1,...,2,1,3,3,1,5,5,1,4,5
2022-06-28,4,5,5,4,2,3,1,4,2,3,...,5,1,4,2,1,5,3,2,1,5


In [15]:
rd3_next_month.loc[rd3_next_month.index=='2022-05-27']

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-05-27,1,1,2,4,5,2,5,4,4,4,...,3,5,3,1,1,1,5,3,4,1


#####Functions
***

In [16]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    # max_date = np.max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [17]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [18]:
def predictions(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      # print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-23] #-21 , -19
      
      Y_train= y.iloc[:-23] #-21 , -19

      print(Y_train.tail(1))
      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      flag=True
      for i in range(22,0,-1): #from 4/01 and later -->predictions
        if(flag):
          first=X.iloc[-(i+1):-i].index
          print('this is the first day of predictions', first)
          flag=False
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        # print(Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      X_validation=X.tail(1) 
      last=X_validation.index
      print('this is last for prediction',last)
      # print(X_validation)
      Y_validation= y.tail(1)
      print(Y_validation.index) 
      probability= model.predict_proba(X_validation)
      print(probability[0])
      tmp.append(probability[0])
      prob[j]=tmp
  return prob , first , last 

In [19]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob : ',prob)
      mask1 = (rd.loc[(rd.index.date>=prob_final['date'][k+1])].index)
      prob1_table = rd.loc[mask1]
      prob1 = prob1_table.shift(-21).head(1)
      print('prob1 : ',prob1)

      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob teleutaio :', prob)
      prob1 = pd.DataFrame(prob_table.iloc[1]).T
      print('prob1 : ',prob1)

      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [20]:
def create_RPS(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

####**For 6 Mar.-1 Apr.** 

#####Features
***

In [21]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-03-01','2022-04-02') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [22]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-03-04') & (stock['ABBV'].index < '2022-04-01')]))
# stock['ABBV']

19


In [23]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-03-02']['Rank'])

Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, dtype: int64
Date
2022-03-02    2
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    2
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    2
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, d

#####Gradient Boosting(baseline)
***

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-03-01    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.25646178 0.0549045  0.2372479  0.14052667 0.31085915]
Date
2022-03-01    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16700058 0.12595881 0.11788762 0.30463099 0.284522  ]
Date
2022-03-01    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-03  159.814133  99.930481  183.961975  110.589905  126.775017   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-03  294.668579  163.559998  252.387024  173.579086  185.944336  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-03  115.603111  116.669998  303.419739  34.615841  89.954269   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-03  509.170898  354.0  225.529037  213.353729  81.485481  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-04  159.038071  99.08387  180.370453  110.371742  128.794907   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-04  294.787476  168.346497  248.984695  173.113022  186.431122  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-04  113.630829  121.860001  293.727295  34.894276  92.495743   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-04  506.617981  349.899994  226.903992  216.233383  81.524696  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-31  146.499664  96.603073  179.417999  113.822716  112.406975   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586754   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753494  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-01  148.213638  97.046082  181.640396  114.437546  113.232841   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-04-01  269.58197  124.253502  220.645218  179.657928  171.468307  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-01  107.137917  90.709999  276.180115  32.527554  99.723053   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-04-01  496.287384  312.450012  207.794388  199.403824  86.45578  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.067451,0.012699,0.437654,0.033784,0.370026,0.790646,0.086423,0.272661,0.106429,0.175519,...,0.384620,0.341455,0.189566,0.403365,0.101286,0.115140,0.150211,0.273198,0.090796,0.142941
2022-03-03,0.094895,0.407957,0.404336,0.078285,0.139862,0.003834,0.418490,0.337655,0.051861,0.203086,...,0.263665,0.427852,0.257363,0.104298,0.392347,0.161910,0.311881,0.172190,0.011499,0.153153
2022-03-04,0.179221,0.014040,0.120102,0.092149,0.261609,0.113549,0.223943,0.120200,0.190952,0.156565,...,0.139988,0.004305,0.079980,0.032673,0.064673,0.261881,0.785337,0.087887,0.246207,0.160646
2022-03-07,0.095695,0.175888,0.156802,0.273647,0.053692,0.213010,0.176636,0.222694,0.076354,0.199110,...,0.157641,0.001598,0.192561,0.134423,0.126443,0.382494,0.431530,0.184948,0.142154,0.195476
2022-03-08,0.155566,0.492696,0.046869,0.183892,0.509457,0.257889,0.416661,0.275291,0.314849,0.314544,...,0.298866,0.331026,0.199644,0.258324,0.165662,0.186130,0.722815,0.141497,0.177015,0.063174
2022-03-09,0.104572,0.574081,0.138593,0.454309,0.446690,0.351978,0.207220,0.134721,0.145225,0.107343,...,0.261699,0.130424,0.002173,0.233065,0.151669,0.434732,0.141957,0.258461,0.110917,0.089754
2022-03-10,0.384356,0.100808,0.114474,0.188633,0.182224,0.291965,0.099152,0.119739,0.360423,0.584460,...,0.097268,0.200515,0.023815,0.605311,0.354295,0.004383,0.224707,0.036546,0.090857,0.039658
2022-03-11,0.347235,0.124639,0.050317,0.023153,0.031077,0.001354,0.404816,0.110165,0.076473,0.267651,...,0.259047,0.177076,0.387542,0.046058,0.573328,0.009808,0.249051,0.288089,0.123187,0.250181
2022-03-14,0.465363,0.638166,0.082630,0.361680,0.181223,0.584393,0.436362,0.430084,0.278946,0.098990,...,0.154309,0.360045,0.004429,0.190012,0.159225,0.089121,0.023791,0.208443,0.600344,0.202598


In [26]:
print(RPS1.mean(axis=1).mean())

0.20804906137097678


In [27]:
print(RPS1.mean(axis=1))

date
2022-03-02    0.218012
2022-03-03    0.256557
2022-03-04    0.169645
2022-03-07    0.182346
2022-03-08    0.214771
2022-03-09    0.195755
2022-03-10    0.195517
2022-03-11    0.173667
2022-03-14    0.221698
2022-03-15    0.168896
2022-03-16    0.215901
2022-03-17    0.201557
2022-03-18    0.214509
2022-03-21    0.202889
2022-03-22    0.202506
2022-03-23    0.208323
2022-03-24    0.190937
2022-03-25    0.225125
2022-03-28    0.229413
2022-03-29    0.223454
2022-03-30    0.227802
2022-03-31    0.196733
2022-04-01    0.249116
dtype: float64


In [28]:
#save results 
RPS1.to_excel('6_Mar_1_Apr-grad_boost(baseline)-real_data.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('6_Mar_1_Apr-PROBABILITIES-grad_boost(baseline)-real_data.xlsx')

#####After feature importances and tuning
***

In [29]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [30]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-03-01    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.11889655 0.19887301 0.21375941 0.21628527 0.25218575]
Date
2022-03-01    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.18561651 0.2146882  0.19135995 0.22470128 0.18363406]
Date
2022-03-01    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-03  159.814133  99.930481  183.961975  110.589905  126.775017   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-03  294.668579  163.559998  252.387024  173.579086  185.944336  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-03  115.603111  116.669998  303.419739  34.615841  89.954269   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-03  509.170898  354.0  225.529037  213.353729  81.485481  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-04  159.038071  99.08387  180.370453  110.371742  128.794907   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-04  294.787476  168.346497  248.984695  173.113022  186.431122  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-04  113.630829  121.860001  293.727295  34.894276  92.495743   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-04  506.617981  349.899994  226.903992  216.233383  81.524696  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-31  146.499664  96.603073  179.417999  113.822716  112.406975   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586754   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753494  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-01  148.213638  97.046082  181.640396  114.437546  113.232841   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-04-01  269.58197  124.253502  220.645218  179.657928  171.468307  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-01  107.137917  90.709999  276.180115  32.527554  99.723053   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-04-01  496.287384  312.450012  207.794388  199.403824  86.45578  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.075861,0.211589,0.254235,0.126440,0.236160,0.300412,0.114650,0.235491,0.065889,0.125355,...,0.127078,0.122615,0.210627,0.247192,0.107988,0.073311,0.126938,0.248277,0.132954,0.101864
2022-03-03,0.139046,0.122006,0.274941,0.073538,0.132285,0.071266,0.244460,0.264680,0.101939,0.121010,...,0.246408,0.254842,0.268595,0.115490,0.264401,0.134003,0.210310,0.113806,0.102423,0.103705
2022-03-04,0.191373,0.110797,0.059547,0.102547,0.236160,0.074460,0.216628,0.074229,0.129823,0.123311,...,0.120475,0.176343,0.114033,0.086932,0.094761,0.223423,0.210310,0.073089,0.238417,0.102603
2022-03-07,0.191373,0.206850,0.110408,0.226038,0.145363,0.265203,0.216628,0.254274,0.101939,0.125867,...,0.079007,0.176343,0.129449,0.087029,0.139194,0.200947,0.221141,0.256869,0.064254,0.131146
2022-03-08,0.201835,0.262592,0.055631,0.123190,0.200837,0.278184,0.116817,0.266620,0.116056,0.140712,...,0.246408,0.122615,0.133992,0.215893,0.132339,0.102930,0.221141,0.241520,0.064859,0.211293
2022-03-09,0.098439,0.119325,0.056905,0.228676,0.229578,0.105396,0.216628,0.071416,0.122786,0.076916,...,0.230264,0.176343,0.071758,0.131214,0.134211,0.225826,0.210310,0.077401,0.132954,0.219024
2022-03-10,0.296918,0.079275,0.252236,0.274145,0.228403,0.104971,0.116817,0.074341,0.283210,0.255426,...,0.120475,0.122615,0.114528,0.221893,0.264401,0.069992,0.140710,0.077651,0.064052,0.224937
2022-03-11,0.296918,0.237379,0.055631,0.073120,0.228403,0.127521,0.138299,0.103099,0.101939,0.128157,...,0.124734,0.176343,0.271188,0.085934,0.264560,0.073311,0.233308,0.241520,0.133598,0.212474
2022-03-14,0.303476,0.245265,0.107041,0.269832,0.200730,0.231819,0.241696,0.137541,0.231775,0.224514,...,0.080112,0.176343,0.071758,0.131214,0.094568,0.073311,0.230397,0.121326,0.272392,0.133781


In [31]:
print(RPS2.mean(axis=1).mean())

0.1585344532578433


In [32]:
print(RPS2.mean(axis=1))

date
2022-03-02    0.161919
2022-03-03    0.163125
2022-03-04    0.151839
2022-03-07    0.153646
2022-03-08    0.155545
2022-03-09    0.155332
2022-03-10    0.166809
2022-03-11    0.156031
2022-03-14    0.161044
2022-03-15    0.154768
2022-03-16    0.155586
2022-03-17    0.163835
2022-03-18    0.159758
2022-03-21    0.157027
2022-03-22    0.161012
2022-03-23    0.158402
2022-03-24    0.155735
2022-03-25    0.155777
2022-03-28    0.158289
2022-03-29    0.162712
2022-03-30    0.162623
2022-03-31    0.155319
2022-04-01    0.160157
dtype: float64


In [33]:
#save results 
RPS2.to_excel('6_Mar_1_Apr-grad_boost(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('6_Mar_1_Apr-PROBABILITIES-grad_boost(with_feature_imp)-real_data.xlsx')

####**For 3 Apr.-29 Apr.**

#####Features
***

In [34]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-04-01','2022-05-02') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [35]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-04-01  108.919998  108.980003  107.320000  108.519997  101.861610
 2021-04-05  109.080002  109.080002  105.400002  106.139999   99.627640
 2021-04-06  105.849998  106.279999  104.879997  105.379997   98.914284
 2021-04-07  105.529999  106.129997  104.169998  105.209999   98.754700
 2021-04-08  105.400002  106.919998  105.349998  106.099998   99.590096
 ...                ...         ...         ...         ...         ...
 2022-04-25  154.839996  156.639999  152.839996  156.300003  154.851547
 2022-04-26  156.440002  159.690002  155.820007  156.179993  154.732651
 2022-04-27  156.559998  159.500000  154.559998  157.619995  156.159302
 2022-04-28  158.360001  158.440002  154.339996  156.309998  154.861450
 2022-04-29  149.559998  149.860001  139.929993  146.880005  145.518845
 
 [273 rows x 5 columns],
 'AEP':                   Ope

In [36]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-04-01') & (stock['ABBV'].index <= '2022-04-29')]))

19


In [37]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(1)-df['Open'])/df['Open'].shift(1)
    df['(high2-high1)/high2']= (df['High'].shift(1)-df['High'])/df['High'].shift(1)
    df['(low2-low1)/low2']= (df['Low'].shift(1)-df['Low'])/df['Low'].shift(1)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(1)-df['Adj Close'])/df['Adj Close'].shift(1)
    df['av_3']=df['Adj Close'].rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High']-df['Low'])/df['High']
    df['close1-open1/close1']=(df['Close']-df['Open'])/df['Close']
    df['RSI']= ta.RSI(np.array(df['Adj Close']),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'], df['Low'], df['Close'], timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close']),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'],timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'],timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'], df['Low'], df['Close'],timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std()
    df['dist_from_mean_10'] = (df['Adj Close'] - (df['Adj Close'].rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'] - (df['Adj Close'].rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'],fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'], fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'],timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-03-29']['Rank'])

Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    3
Name: Rank, dtype: int64
Date
2022-03-29    3
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, d

#####Gradient Boosting (baseline)
***

In [38]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-03-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.02422767 0.0037488  0.38504092 0.57962496 0.00735765]
Date
2022-03-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.85877057 0.01430425 0.02109886 0.08047605 0.02535026]
Date
2022-03-28    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-30  145.518845  97.567833  180.449829  113.287224  109.800034   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-30  263.042816  124.281502  225.644547  179.092697  174.058624  ...   

                   PRU   PYPL          RE        ROL       ROST         UNH  \
Date                                                                          
2022-03-30  105.946739  87.93  271.417877  33.352924  99.047974  505.157806   

                   URI          V        VRSK        XOM  
Date                                                      
2022-03-30  316.519989  212.34761  203.319763  83.573593  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-31  146.499664  96.603073  179.417999  113.822716  112.406975   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586754   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753494  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  112.593056  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  110.718826  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE      

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-29,0.301733,0.180394,0.218138,0.093188,0.144910,0.162711,0.376612,0.357298,0.376236,0.013565,...,0.360369,0.055185,0.136741,0.124243,0.205509,0.018179,0.192525,0.086553,0.232821,0.340851
2022-03-30,0.280461,0.209432,0.214844,0.064127,0.128561,0.001600,0.138851,0.041106,0.091062,0.267230,...,0.283149,0.053344,0.420835,0.324294,0.160799,0.395363,0.476647,0.308108,0.222795,0.031593
2022-03-31,0.093697,0.034010,0.107998,0.092989,0.140946,0.118233,0.001290,0.022001,0.365641,0.006913,...,0.040010,0.388537,0.115513,0.179742,0.427116,0.203179,0.238949,0.327798,0.373937,0.152714
2022-04-01,0.556917,0.278643,0.294746,0.121796,0.003333,0.272844,0.196099,0.395726,0.005723,0.123365,...,0.183664,0.127530,0.135450,0.324095,0.002245,0.278378,0.010553,0.122128,0.317734,0.279042
2022-04-04,0.153127,0.269826,0.073961,0.237287,0.524402,0.329392,0.205895,0.153761,0.016985,0.109724,...,0.043088,0.002403,0.046635,0.143594,0.194944,0.136586,0.325602,0.475342,0.009884,0.139441
2022-04-05,0.070073,0.190866,0.365095,0.203652,0.145809,0.205639,0.207879,0.194232,0.140216,0.340188,...,0.040948,0.133816,0.088858,0.266729,0.302972,0.284721,0.367433,0.512286,0.115786,0.145485
2022-04-06,0.163311,0.331240,0.255138,0.656797,0.334263,0.034896,0.191807,0.169614,0.131013,0.251535,...,0.273697,0.234193,0.096417,0.108341,0.411615,0.018940,0.161261,0.251304,0.184238,0.357699
2022-04-07,0.119569,0.040531,0.007724,0.571209,0.536603,0.121406,0.193205,0.012305,0.303043,0.075385,...,0.206366,0.095553,0.104146,0.372363,0.455472,0.292615,0.195402,0.317983,0.159175,0.096179
2022-04-08,0.144972,0.370504,0.055932,0.197142,0.513804,0.171923,0.118760,0.209382,0.024730,0.154844,...,0.152348,0.081346,0.121164,0.120785,0.029208,0.102417,0.395377,0.423322,0.178062,0.471468


In [39]:
print(RPS1.mean(axis=1).mean())

0.2109865387701118


In [40]:
print(RPS1.mean(axis=1))

date
2022-03-29    0.185918
2022-03-30    0.196286
2022-03-31    0.203480
2022-04-01    0.192503
2022-04-04    0.183856
2022-04-05    0.234095
2022-04-06    0.237276
2022-04-07    0.205057
2022-04-08    0.191792
2022-04-11    0.229468
2022-04-12    0.200558
2022-04-13    0.184836
2022-04-14    0.202551
2022-04-18    0.203431
2022-04-19    0.211214
2022-04-20    0.223071
2022-04-21    0.222297
2022-04-22    0.213602
2022-04-25    0.183059
2022-04-26    0.248093
2022-04-27    0.279115
2022-04-28    0.210194
2022-04-29    0.210938
dtype: float64


In [41]:
#save results 
RPS1.to_excel('4_Apr_29_Apr-grad_boost(baseline)-real_data.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('4_Apr_29_Apr-PROBABILITIES-grad_boost(baseline)-real_data.xlsx')

#####After feature importances and tuning
***

In [42]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [43]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-03-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13292548 0.16815956 0.22705198 0.25212241 0.21974057]
Date
2022-03-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.2036962  0.19128579 0.18262935 0.22972998 0.19265869]
Date
2022-03-28    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-30  145.518845  97.567833  180.449829  113.287224  109.800034   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-30  263.042816  124.281502  225.644547  179.092697  174.058624  ...   

                   PRU   PYPL          RE        ROL       ROST         UNH  \
Date                                                                          
2022-03-30  105.946739  87.93  271.417877  33.352924  99.047974  505.157806   

                   URI          V        VRSK        XOM  
Date                                                      
2022-03-30  316.519989  212.34761  203.319763  83.573593  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-31  146.499664  96.603073  179.417999  113.822716  112.406975   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586754   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753494  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  112.593056  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  110.718826  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE      

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-29,0.300485,0.131772,0.104244,0.226640,0.140285,0.100642,0.200275,0.256620,0.284349,0.124014,...,0.076814,0.151270,0.078644,0.122435,0.225789,0.069174,0.241208,0.126677,0.130773,0.128232
2022-03-30,0.090312,0.131772,0.056097,0.123662,0.211986,0.100642,0.093635,0.264602,0.103274,0.254987,...,0.076850,0.284015,0.121338,0.125427,0.246648,0.288186,0.212506,0.117499,0.130773,0.128232
2022-03-31,0.090312,0.079940,0.104244,0.079339,0.115171,0.102160,0.091281,0.071218,0.128710,0.124014,...,0.123004,0.151270,0.076948,0.240348,0.221957,0.139913,0.241208,0.235943,0.274757,0.211906
2022-04-01,0.300485,0.127913,0.124173,0.078310,0.203859,0.223806,0.183556,0.264602,0.060566,0.120066,...,0.124011,0.118175,0.079823,0.226862,0.132226,0.288186,0.093324,0.121293,0.274757,0.206316
2022-04-04,0.197043,0.112385,0.249256,0.115246,0.229522,0.268480,0.192958,0.108735,0.065982,0.124014,...,0.076086,0.157579,0.217640,0.122435,0.116859,0.099369,0.212506,0.235943,0.053304,0.122707
2022-04-05,0.092177,0.112385,0.056097,0.116268,0.113771,0.136829,0.192958,0.264602,0.101819,0.249589,...,0.122546,0.157579,0.218611,0.226119,0.221755,0.217950,0.212506,0.127984,0.130026,0.206316
2022-04-06,0.092177,0.220519,0.124359,0.256920,0.223203,0.074470,0.192958,0.126672,0.131050,0.124014,...,0.119060,0.126322,0.076948,0.123658,0.121318,0.065725,0.127507,0.251038,0.130026,0.243091
2022-04-07,0.090312,0.119470,0.103861,0.243907,0.210532,0.074836,0.097232,0.262436,0.232900,0.120066,...,0.119221,0.120548,0.076948,0.240348,0.221957,0.069174,0.223962,0.076241,0.107183,0.128039
2022-04-08,0.090312,0.236048,0.057237,0.113656,0.214847,0.074470,0.183556,0.233789,0.131050,0.119793,...,0.078442,0.157579,0.115352,0.226119,0.112291,0.099369,0.212311,0.247869,0.130026,0.211087


In [44]:
print(RPS2.mean(axis=1).mean())

0.1592691718114164


In [45]:
print(RPS2.mean(axis=1))

date
2022-03-29    0.160771
2022-03-30    0.166052
2022-03-31    0.154768
2022-04-01    0.161748
2022-04-04    0.150919
2022-04-05    0.151900
2022-04-06    0.158336
2022-04-07    0.158091
2022-04-08    0.146854
2022-04-11    0.166227
2022-04-12    0.162566
2022-04-13    0.151671
2022-04-14    0.161399
2022-04-18    0.159973
2022-04-19    0.168224
2022-04-20    0.164343
2022-04-21    0.154391
2022-04-22    0.151367
2022-04-25    0.162494
2022-04-26    0.163230
2022-04-27    0.163631
2022-04-28    0.163699
2022-04-29    0.160538
dtype: float64


In [46]:
#save results 
RPS2.to_excel('4_Apr_29_Apr-grad_boost(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('4_Apr_29_Apr-PROBABILITIES-grad_boost(with_feature_imp)-real_data.xlsx')

####**For 1 May-27 May**

#####Features
***

In [47]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-05-01','2022-05-28') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-05-03  112.400002  115.099998  111.589996  114.680000  108.952454
 2021-05-04  114.620003  115.940002  113.050003  113.900002  108.211418
 2021-05-05  114.309998  116.279999  113.949997  115.779999  109.997520
 2021-05-06  115.000000  116.180000  114.160004  116.080002  110.282524
 2021-05-07  116.379997  116.930000  114.959999  115.750000  109.969025
 ...                ...         ...         ...         ...         ...
 2022-05-23  151.649994  152.000000  147.190002  148.029999  146.658188
 2022-05-24  147.029999  151.350006  146.610001  149.110001  147.728180
 2022-05-25  150.360001  152.990005  149.789993  151.960007  150.551773
 2022-05-26  151.630005  153.070007  150.389999  150.570007  149.174652
 2022-05-27  149.520004  150.050003  147.750000  150.000000  148.609924
 
 [272 rows x 5 columns],
 'AEP':                   Ope

In [48]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-04-27') & (stock['ABBV'].index <= '2022-05-27')]))

22


In [49]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-04-27']['Rank'])

Date
2022-04-27    1
Name: Rank, dtype: int64
Date
2022-04-27    1
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    4
Name: Rank, dtype: int64
Date
2022-04-27    5
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    5
Name: Rank, dtype: int64
Date
2022-04-27    4
Name: Rank, dtype: int64
Date
2022-04-27    4
Name: Rank, dtype: int64
Date
2022-04-27    4
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    3
Name: Rank, dtype: int64
Date
2022-04-27    1
Name: Rank, dtype: int64
Date
2022-04-27    3
Name: Rank, dtype: int64
Date
2022-04-27    3
Name: Rank, dtype: int64
Date
2022-04-27    4
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    5
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    2
Name: Rank, dtype: int64
Date
2022-04-27    5
Name: Rank, dtype: int64
Date
2022-04-27    1
Name: Rank, d

#####Gradient Boosting (baseline)
***

In [50]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13939111 0.23682483 0.29070941 0.11308529 0.21998936]
Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.04554649 0.58233856 0.08107909 0.23257187 0.05846399]
Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  112.593056  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  110.718826  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-04-27,0.248384,0.449565,0.215138,0.045362,0.063281,0.107752,0.228283,0.127697,0.178124,0.140037,...,0.151192,0.011398,0.019972,0.594815,0.186936,0.189473,0.278435,0.019618,0.201805,0.535316
2022-04-28,0.376928,0.089078,0.283582,0.150091,0.194965,0.153672,0.581390,0.180812,0.231286,0.108720,...,0.107492,0.661576,0.000764,0.193735,0.331251,0.277961,0.443156,0.409185,0.356608,0.395401
2022-04-29,0.085016,0.337281,0.369598,0.265769,0.131175,0.444469,0.310569,0.001090,0.359986,0.321445,...,0.113183,0.003724,0.189602,0.166849,0.496823,0.151621,0.074335,0.280508,0.331161,0.009908
2022-05-02,0.017067,0.341260,0.179780,0.142357,0.171594,0.113005,0.161372,0.383581,0.258921,0.132564,...,0.002464,0.773706,0.022302,0.314286,0.207606,0.208988,0.246704,0.245933,0.308624,0.186128
2022-05-03,0.261510,0.009620,0.348925,0.273332,0.010684,0.015312,0.023104,0.504634,0.430929,0.026737,...,0.295783,0.004024,0.161209,0.283218,0.103808,0.048959,0.116896,0.025862,0.156501,0.046856
2022-05-04,0.347574,0.241268,0.063243,0.020769,0.134350,0.070087,0.528260,0.478136,0.283452,0.134034,...,0.077201,0.227454,0.315771,0.121555,0.215265,0.356563,0.454002,0.037252,0.162207,0.306199
2022-05-05,0.201849,0.170844,0.047514,0.007164,0.152680,0.062063,0.104265,0.177113,0.212147,0.242984,...,0.001490,0.742452,0.032704,0.245009,0.314704,0.135408,0.336032,0.107028,0.200522,0.006145
2022-05-06,0.446128,0.302761,0.210374,0.419405,0.039818,0.124544,0.196008,0.583976,0.128592,0.064117,...,0.133759,0.158867,0.567845,0.484126,0.352348,0.193879,0.098988,0.117104,0.260647,0.141119
2022-05-09,0.233960,0.245575,0.323553,0.100594,0.084962,0.116062,0.128056,0.311429,0.265422,0.172823,...,0.148892,0.027512,0.102643,0.261066,0.130335,0.192095,0.143167,0.706368,0.024801,0.180347


In [51]:
print(RPS1.mean(axis=1).mean())

0.2006540076712491


In [52]:
print(RPS1.mean(axis=1))

date
2022-04-27    0.193071
2022-04-28    0.234659
2022-04-29    0.197943
2022-05-02    0.214946
2022-05-03    0.155997
2022-05-04    0.209439
2022-05-05    0.168516
2022-05-06    0.223602
2022-05-09    0.241863
2022-05-10    0.212432
2022-05-11    0.211102
2022-05-12    0.202325
2022-05-13    0.207377
2022-05-16    0.185307
2022-05-17    0.233584
2022-05-18    0.173818
2022-05-19    0.187314
2022-05-20    0.186085
2022-05-23    0.246137
2022-05-24    0.164046
2022-05-25    0.175769
2022-05-26    0.189916
2022-05-27    0.199796
dtype: float64


In [53]:
#save results 
#save results 
RPS1.to_excel('2_May_27_May-grad_boost(baseline)-real_data.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('2_May_27_May-PROBABILITIES-grad_boost(baseline)-real_data.xlsx')

#####After feature importances and tuning
***

In [54]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [55]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15811809 0.18082735 0.19320186 0.22625981 0.24159289]
Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.1600734  0.22596625 0.1741926  0.24951201 0.19025574]
Date
2022-04-26    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  112.593056  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  110.718826  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-04-27,0.285983,0.254318,0.122596,0.127009,0.198520,0.141482,0.264574,0.115720,0.128066,0.129903,...,0.072644,0.292149,0.072761,0.253599,0.216004,0.273137,0.231423,0.074129,0.133649,0.282347
2022-04-28,0.151556,0.120287,0.279001,0.108542,0.142850,0.074241,0.264561,0.117203,0.128066,0.129903,...,0.116800,0.292149,0.067302,0.244819,0.219804,0.273137,0.231423,0.125351,0.245583,0.162376
2022-04-29,0.083691,0.111726,0.245449,0.269951,0.218791,0.278022,0.266060,0.115883,0.277950,0.112424,...,0.072644,0.142979,0.134786,0.130096,0.214343,0.067676,0.128502,0.080185,0.253043,0.169280
2022-05-02,0.151556,0.255107,0.120471,0.071719,0.140886,0.074241,0.266043,0.130994,0.128402,0.131404,...,0.248201,0.291715,0.067302,0.244819,0.111964,0.273137,0.135487,0.125351,0.131318,0.268358
2022-05-03,0.196527,0.111373,0.253795,0.232413,0.218791,0.141482,0.195996,0.263279,0.128402,0.112424,...,0.248201,0.142979,0.113887,0.223512,0.090155,0.124805,0.233255,0.079271,0.116320,0.183435
2022-05-04,0.284602,0.122141,0.107818,0.071665,0.142850,0.095790,0.266060,0.243927,0.128402,0.112072,...,0.120176,0.130177,0.216416,0.088384,0.214343,0.111883,0.231349,0.077456,0.059041,0.162376
2022-05-05,0.200717,0.122208,0.105032,0.122162,0.142561,0.073989,0.195996,0.253172,0.061231,0.077525,...,0.061459,0.292149,0.071725,0.117509,0.214343,0.105221,0.235082,0.111747,0.116320,0.169280
2022-05-06,0.200717,0.121173,0.122596,0.233562,0.218791,0.140916,0.090359,0.263279,0.112731,0.100330,...,0.070724,0.134878,0.274533,0.151088,0.254644,0.111883,0.208006,0.125351,0.245643,0.183435
2022-05-09,0.146605,0.081240,0.105178,0.115997,0.218791,0.141482,0.195996,0.117344,0.129637,0.239645,...,0.118502,0.142979,0.113349,0.117509,0.126364,0.219246,0.107890,0.237857,0.059041,0.101740


In [56]:
print(RPS2.mean(axis=1).mean())

0.15875668329411152


In [57]:
print(RPS2.mean(axis=1))

date
2022-04-27    0.163988
2022-04-28    0.163627
2022-04-29    0.158742
2022-05-02    0.168775
2022-05-03    0.148566
2022-05-04    0.162650
2022-05-05    0.150877
2022-05-06    0.163909
2022-05-09    0.158836
2022-05-10    0.150341
2022-05-11    0.153874
2022-05-12    0.162936
2022-05-13    0.166247
2022-05-16    0.152517
2022-05-17    0.166212
2022-05-18    0.149836
2022-05-19    0.157329
2022-05-20    0.162845
2022-05-23    0.166573
2022-05-24    0.150315
2022-05-25    0.152090
2022-05-26    0.159548
2022-05-27    0.160769
dtype: float64


In [58]:
#save results 
RPS2.to_excel('2_May_27_May-grad_boost(with_feature_imp)-real_data.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('2_May_27_May-PROBABILITIES-grad_boost(with_feature_imp)-real_data.xlsx')